In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow_datasets as tfds

In [ ]:
train_ds, test_ds = tfds.load('mnist', split=['train', 'test'], as_supervised=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
train_data = []
train_labels = []

for example, label in train_ds:
  train_data.append(example.numpy())
  train_labels.append(label.numpy())


test_data = []
test_labels = []

for example, label in test_ds:
  test_data.append(example.numpy())
  test_labels.append(label.numpy())

train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [ ]:
print(train_data.shape, train_labels.shape)
print(test_data.shape, test_labels.shape)

(60000, 28, 28, 1) (60000,)
(10000, 28, 28, 1) (10000,)


In [ ]:
train_data = train_data / 255
test_data = test_data / 255

In [ ]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('accuracy') > 0.97:
      print('Stopping training')
      self.model.stop_training = True

callback = MyCallback()

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(64, 3, activation='relu', input_shape=[28,28,1]),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(train_data, train_labels, epochs=3, callbacks=[callback])

Epoch 1/3
1875/1875 [==============================] - 84s 44ms/step - loss: 0.1436 - accuracy: 0.9562
Epoch 2/3
1875/1875 [==============================] - 83s 44ms/step - loss: 0.0451 - accuracy: 0.9865
Stopping training


In [ ]:
def predict(model, data, true):
  i = random.randint(0,1000)
  true = true[i]
  data = np.array(data)
  pred = model.predict(data[i].reshape(1, 28, 28, 1))
  print('prediction: ', pred.argmax())
  print('Real: ', true)
  if pred.argmax() - true == 0:
    print('Correct!')
  else:
    print('Incorrect!')

In [ ]:
predict(model=model, data=test_data, true=test_labels)

prediction:  8
Real:  8
Correct!
